In [1]:
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

2023-03-14 13:40:49.664915: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-14 13:40:49.775699: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-14 13:40:49.781084: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-14 13:40:49.781098: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
data = pd.DataFrame({
    "id": [7058, 7059, 7072, 7054],
    "name": ['sravan', 'jyothika', 'harsha', 'ramya'],
    "subjects": ['java', 'python', 'html/php', 'php/js']
})

In [6]:
data.iloc[:,0]

0    7058
1    7059
2    7072
3    7054
Name: id, dtype: int64

In [ ]:
[7058, 7059, 7072, 7054]

# Load data

In [35]:
os.getcwd()

csv_path = os.path.join('..', 'inflation-forecasting', 'raw_data')

df = pd.read_csv(os.path.join(csv_path,'data_final.csv'), index_col=0)

In [36]:
countries  = df['country_id'].unique()

df_us = df[df['country_id'] == "USA"]
df_de = df[df['country_id'] == "DE"]
df_nl = df[df['country_id'] == "NL"]
df_it = df[df['country_id'] == "IT"]
df_fr = df[df['country_id'] == "FR"]
df_es = df[df['country_id'] == "ES"]

# Inflation (CPI)

In [37]:
config = {'displayModeBar': False}

fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=("United States", "Germany", "Netherlands", "Italy", "France", "Spain"))

fig.add_trace(go.Scatter(x = df_us["year"],
                         y = df_us["cpi"],
                         name = "United States"),
              row=1, col=1)

fig.add_trace(go.Scatter(x = df_de["year"],
                         y = df_de["cpi"],
                         name = "Germany"),
              row=1, col=2)

fig.add_trace(go.Scatter(x = df_nl["year"],
                         y = df_nl["cpi"],
                         name = "Netherlands"),
              row=1, col=3)

fig.add_trace(go.Scatter(x = df_it["year"],
                         y = df_it["cpi"],
                         name = "Italy"),
              row=2, col=1)

fig.add_trace(go.Scatter(x = df_fr["year"],
                         y = df_fr["cpi"],
                         name = "France"),
              row=2, col=2)

fig.add_trace(go.Scatter(x = df_es["year"],
                         y = df_es["cpi"],
                         name = "Spain"),
              row=2, col=3)

fig.update_layout(
    title = "CPI",
    autosize=False,
    width = 890,
    height = 600,
  legend = dict(
        xanchor = "center",
        yanchor = "top",
        y = -0.2,
        x = 0.5,
        orientation = 'h'
  )
)

fig.show(config=config)

# Core inflation (CCPI)

In [38]:
config = {'displayModeBar': False}

fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=("United States", "Germany", "Netherlands", "Italy", "France", "Spain"))

fig.add_trace(go.Scatter(x = df_us["year"],
                         y = df_us["ccpi"],
                         name = "United States"),
              row=1, col=1)

fig.add_trace(go.Scatter(x = df_de["year"],
                         y = df_de["ccpi"],
                         name = "Germany"),
              row=1, col=2)

fig.add_trace(go.Scatter(x = df_nl["year"],
                         y = df_nl["ccpi"],
                         name = "Netherlands"),
              row=1, col=3)

fig.add_trace(go.Scatter(x = df_it["year"],
                         y = df_it["ccpi"],
                         name = "Italy"),
              row=2, col=1)

fig.add_trace(go.Scatter(x = df_fr["year"],
                         y = df_fr["ccpi"],
                         name = "France"),
              row=2, col=2)

fig.add_trace(go.Scatter(x = df_es["year"],
                         y = df_es["ccpi"],
                         name = "Spain"),
              row=2, col=3)

fig.update_layout(
    title = "CCPI",
    autosize=False,
    width = 890,
    height = 600,
  legend=dict(
        xanchor = "center",
        yanchor = "top",
        y = -0.2,
        x = 0.5,
        orientation = 'h'
  )
)

fig.show(config=config)

# Load the model

In [39]:
model_usa = load_model("models/model_ccpi_usa.h5")

# Prepare data for LSTM + forecast function

In [40]:
def future_forecasting(data_inflation, model, mb=12, mf=12):
    '''
    Returns the future forecasting of the model. Please select the dataset and model you want to use.
    for mb, select the number of months you want to look back to make a prediction.
    for mf, select the number of months you want the prediction to look forward.
    '''
    ccpi = data_inflation.to_numpy()
    scaler = MinMaxScaler(feature_range = (0, 1))
    dataset = scaler.fit_transform(ccpi.reshape(-1,1))
    results = []
    x_input = dataset[-mb:].reshape(1, mb, 1)
    for num in range(mf):
        forecast = model.predict(x_input)
        results.append(forecast[0][0])
        x_input = np.roll(x_input, -1, axis=1)
        x_input[0][-1] = forecast
    results = scaler.inverse_transform(np.array(results).reshape(-1, 1)) # reshape dataset to 2D array
    results = np.array(results).reshape(-1, 1)
    return results

In [41]:
df_us.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 757 entries, 0 to 756
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   year        757 non-null    object 
 1   cpi         757 non-null    float64
 2   ccpi        757 non-null    float64
 3   indprod     757 non-null    float64
 4   unemp       757 non-null    float64
 5   intrate     757 non-null    float64
 6   m3          744 non-null    float64
 7   oil_price   757 non-null    float64
 8   country_id  757 non-null    object 
 9   country     757 non-null    object 
dtypes: float64(7), object(3)
memory usage: 65.1+ KB


In [42]:
def filter(data, country = "USA", case="ccpi"):
    data_country = data[data['country_id'] == country]
    return data_country[case]

def scaling(data):
    scaler = MinMaxScaler(feature_range = (0, 1))
    scaler.fit(data.to_numpy().reshape(-1,1))
    return scaler

def preprocess(data_inflation, scalar):
    ccpi = data_inflation.to_numpy()
    ccpi_trans = scalar.transform(ccpi)
    return ccpi_trans

In [43]:
df_us_app = filter(df, "USA")

In [44]:
scaler = MinMaxScaler(feature_range = (0, 1))
scaler.fit(df_us_app.to_numpy().reshape(-1,1))

MinMaxScaler()

In [45]:
ccpi = df_us_app.to_numpy().reshape(-1,1)
ccpi_trans = scaler.transform(ccpi)

In [46]:
ccpi_trans.shape

(757, 1)

# Plot function

In [47]:
def plot_function(data_country, data_forecast_country, user_input):
    
    first_date = data_country['year'].min()
    
    last_date = pd.to_datetime(data_country['year'].max()) + pd.offsets.MonthBegin(1)
    
    country_name = data_country["country"].unique()[0]
    
    t = pd.date_range(last_date, periods = user_input, freq='MS')
    forecast_date = t.max()
    out_forecast_df = pd.DataFrame([[x, y] for x, y in zip(t, data_forecast_country)], columns=["year", "Forecast"])
    
    out_forecast_df['Forecast'] = out_forecast_df['Forecast'].apply(lambda x: np.ravel(x)[0])
    
    config = {'displayModeBar': False}
    fig = go.Figure()
    df_short = data_country.loc[data_country["year"] >= "2015-01-01"]

    fig.add_trace(go.Scatter(x = df_short["year"], 
                             y = df_short['ccpi'],
                             line_color = "blue",
                             name = "Core CPI", 
                             mode = "lines"))

    fig.add_trace(go.Scatter(x = out_forecast_df["year"], 
                             y = out_forecast_df['Forecast'], 
                             name = "Forecasted Core CPI",
                             line_dash = "dash", 
                             line_color = "black",
                             mode = "lines"))

    fig.add_vrect(x0 = last_date, 
                  x1 = forecast_date,  
                  fillcolor = "grey", 
                  opacity = 0.25, 
                  line_width = 0)

    fig.update_layout(
        title = f"Out-of-sample forecast until {forecast_date.strftime('%B %Y')} for {country_name}", 
        xaxis_title = "", 
        yaxis_title = "Monthly y-o-y percentage change",
        autosize=False,
        hoverlabel_namelength=-1,
        width=890,
        height=600,
       legend=dict(
             xanchor = "center",
             yanchor = "top",
             y = -0.2, 
             x = 0.5,
             orientation = 'h'
       ),
      xaxis=dict(
          dtick='M12',
          tickangle=45,
          tickfont=dict(size=14)
        ),
      yaxis=dict(
          tickfont=dict(size=14)
        )
    )
    
    fig.update_layout(
    plot_bgcolor='white')
    
    fig.update_xaxes(
        mirror=True,
        ticks='outside',
        showline=True,
        linecolor='black',
        gridcolor='lightgrey'
    )
    fig.update_yaxes(
        mirror=True,
        ticks='outside',
        showline=True,
        linecolor='black',
        gridcolor='lightgrey')

    fig.show(config=config)

# Test of the plot function

- The function takes two arguments:
    - the dataframe of a single country, e.g. *df_us*
    - the dataframe of the forecast created with the *future_forecasting* function
- The function takes care of the time range and of the shaded area
- If we decide to forecast CPI and not CCPI, we have to change the name manually
- The function use the starting date "2015-01-01" but it can be manually changed

This is the number of month of the forecast input by the user in the **streamlit app**.

In [48]:
user_input = 48

In [49]:
inf_for_us = future_forecasting(data_inflation = df_us["ccpi"], 
                                model = model_usa, 
                                mb = 12, 
                                mf = user_input)

1/1 [==============================] - 0s 20ms/step


In [50]:
predictions = inf_for_us
# predictions_flat = predictions.flatten()
# predictions_list = list(predictions_flat)
# print(predictions_list)
# return {'predictions': predictions_list}

In [51]:
plot_function(data_country = df_us, 
              data_forecast_country = inf_for_us,
              user_input = user_input)

In [ ]:
# https://note.nkmk.me/en/python-pandas-dtype-astype/

In [62]:
df = pd.DataFrame([10.54546, 15.356456, 20.76744], dtype="f2", columns=["a"])

In [63]:
df

,a
0,10.546875
1,15.359375
2,20.765625


In [65]:
pd.DataFrame(df_us['ccpi'], dtype='f2')

0      2.005859
1      2.341797
2      2.000000
3      2.000000
4      1.661133
         ...   
752    6.644531
753    6.300781
754    5.972656
755    5.703125
756    5.546875
Name: ccpi, Length: 757, dtype: float16

In [66]:
import pandas as pd

data = {'ccpi': {'0': None, '1': None, '2': None, '3': None, '4': None, '5': None, '6': None, '7': None, '8': None, '9': None, '10': None, '11': None, '12': None, '13': None, '14': None, '15': None, '16': None, '17': None, '18': None, '19': None, '20': None, '21': None, '22': None, '23': None, '24': None, '25': None, '26': None, '27': None, '28': None, '29': None, '30': None, '31': None, '32': None, '33': None, '34': None, '35': None, '36': None, '37': None, '38': None, '39': None, '40': None, '41': None, '42': None, '43': None, '44': None, '45': None, '46': None, '47': 2.099609375, '48': 1.900390625, '49': 0.5, '50': 1.599609375, '51': 2.400390625, '52': 2.5, '53': 2.400390625}}

df = pd.DataFrame.from_dict(data['ccpi'], orient='index')


In [67]:
df

,0
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
5,NaN
6,NaN
7,NaN
8,NaN
9,NaN
